# CS295/395: Secure Distributed Computation
## In-Class Exercise, week of 10/17/2022

## References:
- [The Paillier Cryptosystem](https://en.wikipedia.org/wiki/Paillier_cryptosystem)
- [The ElGamal Cryptosystem](https://en.wikipedia.org/wiki/ElGamal_encryption)

In [14]:
import galois
import random
bits = 128

## Question -2

Describe the main idea behind Yao's garbled circuit protocol.

- Constant Round Complexity??
- 2 Parties: Garbler and the Evaluator
- Garbler creates garbled tables
  - AND truth table
  - 1. G generates random wire labels (could be called wire keys b/c they function as encryption keys)
    - $w_i^0$ = random key, $w_i^1$ = random key, so two keys for each input wire
  - 2. Replace all truth table values with wire labels (garbled and table below)

| $w_i$  |     $w_j$     |  $w_k$ |
|----------|-------------|------|
| $w_i^0$ |  $w_j^0$ | $w_k^0$ |
| $w_i^0$ |    $w_j^1$   |   $w_k^0$ |
| $w_i^1$ | $w_j^0$ |    $w_k^0$ |
| $w_i^1$ | $w_j^1$ | $w_k^1$ |

  - 3. Encrypt output wire labels using the two input labels as the encryption keys

| $w_i$  |     $w_j$     |  $w_k$ |
|----------|-------------|------|
| $w_i^0$ |  $w_j^0$ | $enc(w_k^0)$ |
| $w_i^0$ |    $w_j^1$   |   $enc(w_k^0)$ |
| $w_i^1$ | $w_j^0$ |    $enc(w_k^0)$ |
| $w_i^1$ | $w_j^1$ | $enc(w_k^1)$ |

  - 4. Delete first two columns and shuffle output column

- Evaluator knows
  - Garbled tables
  - wire labels for input wires
    - $w_i^*$ and $w_j^*$ are the two wire labels for i and j. Evaluator doesnt know the values, only knows the label
  - 1. decrypt each row of the table
  - 2. All but one will fail => this will output the active wire label for the gates output $w_k^*$. Values of gate still unknown to evaluator

### Gist
Two parties: Garbler (G) and evaluator (E)

Input section:
- Inputs G knwosn:
  ## YOU MISSED THIS PART

Garbled table generation
- For each wire in the circuit, G generates two wire labels, one for the case wire = 0, one for the case wire = 1
- For each gate in the circuit, G generates a garbled table
  - G generates the truth table for the gate
  - G replaces values with wire lables
  - G encrypts output wires with input wire labels
  - G deletes input wire columns, and shuffles the output rows
  - This is the garbled table
  - Key properties: (1) Garbled tables appear random (2) given two input wire labels, it is possible to decrypt exactly one row of the garbled table
- G sends the garbled tables to E

Circuit evaluation
- Assume E knows the active wire labels for each input to the circuit
- For each gate in the circuit:
  - Use active wire labels as keys to try and decrypt every row of the table
  - The decryption result is equal to the active wire label for the gates output
- After evaluating all gates, E knows active wire labels for all wire in the circuit

Output
- E sends active wire lables for all output wire of the circuit
- G knowns the values corresponding for each wire label so they know the output. G sends these values to E


## Question -1

What are the primary advantages and disadvantages of Yao's garbled circuit protocol?

- It has constant round time complexity, *regardless of circuit size*
- 

## Question 1

Implement key generation for the Paillier cryptosystem.

$enc(x,pk) + enc(y,pk) = enc(x+y,pk)$

In [70]:
def invmod(x, m):
    """Returns x^-1 mod m."""
    gcd, s, t = galois.egcd(x,m) # extended Euclidean algorithm returns gcd, x, y from as + bt = gcd(a,b)
    return s

def keygen(bits):
    """Generates keys with `bits`-bits of security. Returns a pair: (secret key, public key)."""
    p = galois.random_prime(bits//2)
    q = galois.random_prime(bits//2)

    n = p*q

    g = n + 1
    lamb = (p-1)*(q-1)
    mu = invmod(lamb,n)

    pk = (n,g)
    sk = (lamb,mu)

    return (sk, pk)


sk, pk = keygen(32)

## Question 2

Implement encryption for the Pallier cryptosystem.

In [71]:
def encrypt(m, pk):
    """Encrypts the message `m` with public key `pk`."""
    n, g = pk

    # select random r where 0 < r <= n
    r = random.randint(1, n)
    # compute c = g^m * r^n mod n^2
    c = pow(g, m, n**2) * pow(r, n, n**2) % n**2

    return c

encrypt(5, pk)

4550912484583152124

## Question 3

Implement decryption for the Pallier cryptosystem.

In [72]:
def decrypt(c, sk, pk):
    """Decrypts the ciphertext `c` using secret key `sk` and public key `pk`."""
    n,g = pk
    lamb,mu = sk

    def L(x):
        return (x-1) // n

    m = L(pow(c, lamb, n**2)) * mu % n
    return m

In [73]:
# TEST CASE
sk, pk = keygen(128)
ciphertext = encrypt(25, pk)
plaintext = decrypt(ciphertext, sk, pk)
plaintext

25

## Question 4

Define addition for ciphertexts.

In [74]:
def e_add(c1, c2, pk):
    """Add one encrypted integer to another"""
    n, g = pk

    return c1 * c2 % n**2

In [75]:
# TEST CASE
sk, pk = keygen(128)
ct1 = encrypt(50, pk)
ct2 = encrypt(10, pk)
ct3 = e_add(ct1, ct2, pk)

assert decrypt(ct3, sk, pk) == 60

## Question 5

Define addition-by-constant for ciphertexts.

In [76]:
def e_add_const(c, m, pk):
    """Add constant n to an encrypted integer"""
    n, g = pk

    # c * g**m is equivalent to adding c's message to m
    # roughly equivalent to encrypting c, then adding to m
    return c * pow(g, m, n**2) % n**2

In [77]:
# TEST CASE
sk, pk = keygen(128)
ct1 = encrypt(50, pk)
ct3 = e_add_const(ct1, 10, pk)

assert decrypt(ct3, sk, pk) == 60

## Question 6

Define multiplication-by-constant for ciphertexts.

In [78]:
def e_mul_const(c, m, pk):
    """Multiplies an ancrypted integer by a constant"""
    n, g = pk
    
    # we want to add c to itself m times
    # we can do addition my multiplying c to itself, so c^m is going to be the same as adding c to itself m times
    return pow(c, m, n**2)

In [79]:
# TEST CASE
sk, pk = keygen(128)
ct1 = encrypt(50, pk)
ct3 = e_mul_const(ct1, 3, pk)

assert decrypt(ct3, sk, pk) == 150

In [81]:
sk,pk = keygen(128)
encrypt(0, pk)

87421028552613382048419321503530973445672555391867731948636373866812249749818

## Question 7

Implement key generation, encryption, and decryption for the ElGamal cryptosystem.

In [62]:
def keygen(q):
    # Public key: (G, q, g, h)
    # (G is implicit here)
    # Private key: x

    g = random.randint(2, q)
    x = random.randint(1, q-1)

    h = pow(g, x, q)

    # public ket is (G,q,g,h) but everyone knows G so
    pk = (q,g,h)
    sk = x

    return sk,pk

keygen(2**32-1)

(272106553, (4294967295, 2882424427, 4046063887))

In [63]:
def encrypt(msg, pk):
    q, g, h = pk
    
    y = random.randint(1, q-1)
    s = pow(h, y, q)
    c1 = pow(g, y, q)
    c2 = (msg * s) % q

    return c1, c2

In [64]:
def decrypt(c, x, pk):
    c1, c2 = c
    q, g, h = pk

    s = pow(c1, x, q)
    s_inv = invmod(s, q)

    return c2 * s_inv % q

In [65]:
# TEST CASE
q = 2**127 - 1

sk, pk = keygen(q)
c = encrypt(50, pk) 

decrypt(c, sk, pk)

50

## Question 8

Implement multiplication of ElGamal ciphertexts.

In [66]:
def e_mult(c1, c2, pk):
    q, g, h = pk

    c1_1, c1_2 = c1
    c2_1, c2_2 = c2

    # Piecewise multiplication
    return (c1_1 * c2_1 % q, c1_2 * c2_2 % q)

In [67]:
# TEST CASE
sk, pk = keygen(q)
c1 = encrypt(5, pk)
c2 = encrypt(10, pk)
c3 = e_mult(c1, c2, pk)

assert decrypt(c3, sk, pk) == 50

## Question 9

Describe the differences between the Pallier and ElGamal cryptosystems.

In Pallier, mutliplication => addition
In ElGamal, mult => mult

The big difference is which homomorphism you get.